## DTWDoc2vec: TFIDF Weighted Doc2vec (Delta form, Scaled idf)

In [1]:
import math
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def l2_normalize(x):
    """
    L2 Normalization
    """
    # Calculate the modulus of x
    x_norm = math.sqrt(sum([xi**2 for xi in x]))
    # Return the normalized vector
    return [xi / x_norm for xi in x]

def tfidf_delta_vectorizer(corpus, smooth_idf=True):
    """
    Calculate the TFIDF_delta value
    """
    vectorizer = CountVectorizer()
    # Count the number of times each word appears in all documents
    X = vectorizer.fit_transform(corpus)
    # Get all text keywords in the bag of words
    words = vectorizer.get_feature_names_out()
    # Number of documents
    n = X.shape[0]
    # Matrix for storing TFIDF results
    tfidf_matrix = []
    # Iterate each document and calculate the TFIDF value
    for i in range(n):
        # Get the vector representation of the i-th document
        row = X[i].toarray()[0]
        # List of storing delta values
        delta_scores = []
        # List of storing idf values
        idf_scores = []
        # List of storing TFIDF values
        tfidf_scores = []
        # Iterate each keyword and calculate the TFIDF value
        for j, word in enumerate(words):
            # Calculate delta value (delta = tf - tf_centre)
            delta = row[j] / sum(row) - X[:, j].sum() / X.sum()
            # Calculate df value
            df = X[:, j].count_nonzero()
            # Calculate idf value, with smooth_idf (default) or without smooth_idf
            if smooth_idf:
                idf = math.log((1 + n) / (1 + df)) + 1
            else:
                idf = math.log(n / df) + 1
            # Store delta value in a list
            delta_scores.append(delta)
            # Store idf value in a list
            idf_scores.append(idf)
        # Min-max normalize idf to (0, 1)
        for j, idf in enumerate(idf_scores):
            idf_scores[j] = (idf - min(idf_scores)) / (max(idf_scores) - min(idf_scores))
        # Calculate the TFIDF_delta value
        for j in range(len(delta_scores)):
            tfidf_scores.append(delta_scores[j] * idf_scores[j])
        # L2 normalize the tfidf_scores vector: l2_normalize(tfidf_scores)
        # Store the list of TFIDF values for each document in the matrix
        tfidf_matrix.append(l2_normalize(tfidf_scores))
    return words, np.array(tfidf_matrix)

In [2]:
documents = [
    "The quick brown fox jumps over the lazy dog",
    "The quick brown fox is very agile",
    "The quick brown fox is very smart",
    "The quick brown fox is very quick"
]

In [3]:
words, tfidf_delta_scores = tfidf_delta_vectorizer(documents)
print(words)
print(tfidf_delta_scores)

['agile' 'brown' 'dog' 'fox' 'is' 'jumps' 'lazy' 'over' 'quick' 'smart'
 'the' 'very']
[[-0.15976266 -0.          0.37277955 -0.05558052 -0.30592331  0.37277955
   0.37277955  0.37277955 -0.13895131 -0.15976266  0.21769408 -0.47928799]
 [ 0.76135244  0.         -0.23171596  0.03454829  0.19015881 -0.23171596
  -0.23171596 -0.23171596 -0.08637071 -0.23171596 -0.13531642  0.29792052]
 [-0.23171596  0.         -0.23171596  0.03454829  0.19015881 -0.23171596
  -0.23171596 -0.23171596 -0.08637071  0.76135244 -0.13531642  0.29792052]
 [-0.28673143  0.         -0.28673143  0.04275096  0.23530752 -0.28673143
  -0.28673143 -0.28673143  0.53438698 -0.28673143 -0.16744409  0.36865469]]


## Comparison with sklearn results

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Creating a TF-IDF vectorizer by TfidfVectorizer()
vectorizer_tfidf = TfidfVectorizer()

In [5]:
# Vectorize the data set
vectorized_documents = vectorizer_tfidf.fit_transform(documents)
vectorized_documents_list = vectorized_documents.toarray()

In [6]:
vectorized_documents_list

array([[0.        , 0.21472541, 0.41147631, 0.21472541, 0.        ,
        0.41147631, 0.41147631, 0.41147631, 0.21472541, 0.        ,
        0.42945081, 0.        ],
       [0.58680608, 0.30621975, 0.        , 0.30621975, 0.37455072,
        0.        , 0.        , 0.        , 0.30621975, 0.        ,
        0.30621975, 0.37455072],
       [0.        , 0.30621975, 0.        , 0.30621975, 0.37455072,
        0.        , 0.        , 0.        , 0.30621975, 0.58680608,
        0.30621975, 0.37455072],
       [0.        , 0.3163518 , 0.        , 0.3163518 , 0.38694366,
        0.        , 0.        , 0.        , 0.63270359, 0.        ,
        0.3163518 , 0.38694366]])